In [ ]:
import datetime
import pandas
import pickle
#have to install this (pip install boltons)
import boltons.setutils as bs
%pylab inline

In [ ]:
# function to convert tec date, time string to python datetime obj
def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
    if row["timeStr"] < 10:
        currTimeStr = "000" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 100:
        currTimeStr = "00" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 1000:
        currTimeStr = "0" + str( int( row["timeStr"] ) )
    else:
        currTimeStr = str( int( row["timeStr"] ) )  
    #print datetime.datetime.strptime( currDateStr+ ":" + currTimeStr, "%Y%m%d:%H%M" )
    return datetime.datetime.strptime( currDateStr+ ":" + currTimeStr, "%Y%m%d:%H%M" )

In [ ]:
#loading the TEC data into pandas dataframe
#initializing the columns in the table
inpColList = [ "dateStr", "timeStr", "Mlat", "Mlon", "med_tec", "dlat", "dlon" ]
inpFile = "/home/sneha/GSoC_2018/SpaceVT/dataset/med_filt_tec/tec-medFilt-20150107.txt"
medFiltTECDF = pandas.read_csv(inpFile, delim_whitespace=True, header=None, names=inpColList)
medFiltTECDF["date"] = medFiltTECDF.apply(convert_to_datetime, axis=1)

In [ ]:
#selecting the longitudes in the north-american sector
longitudes = list(range(250, 362, 2)) + list(range(0, 35, 2))
print longitudes
#selecting only those rows whose longitudes is in the range of north-american sector
selectedmedFiltTECDF = medFiltTECDF[medFiltTECDF["Mlon"].isin(longitudes)]
time_date = list(medFiltTECDF["date"])

In [ ]:
times = list(bs.IndexedSet([date for date in time_date]))

In [ ]:
print len(times)

In [ ]:
#d = times[14]
#print d
#print type(d)
#selected = medFiltTECDF[(medFiltTECDF["Mlat"]==12) & (medFiltTECDF["Mlon"].isin(longitudes)) & (medFiltTECDF["date"]==d)]
#print selected            

In [ ]:
#initializing the datetime variable which will be used for selecting values from the table
tec_maps = []
for j in times:
    date = j
    print date
    #selecting those rows where the datetime gives the time with the longitudes are in the given range and latitude = 11
    check_tec_map = medFiltTECDF[(medFiltTECDF["date"]==date)]
    if(len(check_tec_map) >0):
        #populating the TEC maps in a matrix format. The rows corresponds to latitudes and columns corresponds to longitudes
        #Number of latitdues = (89-11) + 1 = 79, number of longitudes = (34-0)/2 + (360-250)/2 + 1 = 73. (number of longitudes is divided by 2 as the degree of longitudes is 2)
        #values in the TEC maps are stored from (250 to 360) then (0 to 34)
        tec_map = np.zeros((75,73,1))
        longi = list(range(0, 35, 2))
        for i in range(0, 75):
            selected = medFiltTECDF[(medFiltTECDF["Mlat"]==(i+15)) & (medFiltTECDF["Mlon"].isin(longitudes)) & (medFiltTECDF["date"]==date)]
            tec = []
            for index, row in selected.iterrows():
                if(int(row["Mlon"]) in longi):
                    j = int(row["Mlon"])/2 + 55
                else:
                    j = ((int(row["Mlon"]) - 250)/2)
                tec_map[i][j] = [float(row["med_tec"])]
        print tec_map.shape
        tec_maps.append(tec_map)
        
tec_maps = np.asarray(tec_maps)
print tec_maps.shape

In [ ]:
#saving the tec_maps in a pickle file
f = open('tec_maps7.pkl','w')
pickle.dump(tec_maps, f)
f.close()

f=open('time_date7.pkl','w')
pickle.dump(times, f)
f.close()